In [2]:
!pip install emoji

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=ac9aecc5be7db2567b747a80afb2c316c2261f784a495415b63f06c2ad459ec9
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [77]:
import tensorflow as tf
import keras
from keras import layers
import pandas as pd
import numpy as np
import re
import emoji
from emoji import emojize

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_emoji.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_emoji.csv")

In [7]:
train.head()

,Sentences,Emojis
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [8]:
emoji_dictionary = {0: "\u2764\uFE0F",
                    1: ":baseball:",
                    2: ":smile:",
                    3: ":disappointed:",
                    4: ":fork_and_knife:"}

In [9]:
filepath = "/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt"

word_to_index = {}
index_to_word = {}
word_to_vec_map = {}
words = []

with open(filepath, "r") as f:
    for i, line in enumerate(f, 1):
        line = line.strip().split()
        word = line[0]
        words.append(word)
        index_to_word[i] = word
        word_to_index[word] = i
        word_to_vec_map[word] = np.array(line[1:], dtype=np.float64)

print(len(word_to_vec_map))

400000


In [10]:
word_to_vec_map["man"]

array([-0.094386,  0.43007 , -0.17224 , -0.45529 ,  1.6447  ,  0.40335 ,
       -0.37263 ,  0.25071 , -0.10588 ,  0.10778 , -0.10848 ,  0.15181 ,
       -0.65396 ,  0.55054 ,  0.59591 , -0.46278 ,  0.11847 ,  0.64448 ,
       -0.70948 ,  0.23947 , -0.82905 ,  1.272   ,  0.033021,  0.2935  ,
        0.3911  , -2.8094  , -0.70745 ,  0.4106  ,  0.3894  , -0.2913  ,
        2.6124  , -0.34576 , -0.16832 ,  0.25154 ,  0.31216 ,  0.31639 ,
        0.12539 , -0.012646,  0.22297 , -0.56585 , -0.086264,  0.62549 ,
       -0.0576  ,  0.29375 ,  0.66005 , -0.53115 , -0.48233 , -0.97925 ,
        0.53135 , -0.11725 ])

In [11]:
sequence_length = 10

def tokenizer(sentences):
    sequences = np.zeros((len(sentences), sequence_length), dtype=np.int32)
    
    for i, sentence in enumerate(sentences):
        #sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
        for j, word in enumerate(sentence.split()):
            if j == sequence_length:
                break
            if word in words:
                sequences[i][j] = word_to_index[word]
            else:
                sequences[i][j] = 0

    
    return sequences

In [12]:
tokenizer(["Let us go outside and play a game"])

array([[  0,  96, 243, 588,   6, 283,   8, 187,   0,   0]], dtype=int32)

In [29]:
X_train = tokenizer(train["Sentences"])
y_train = train["Emojis"]
X_test = tokenizer(test["Sentences"])
y_test = test["Emojis"]

In [30]:
vocab_len = len(word_to_index) + 1
emb_dim = word_to_vec_map["word"].shape[0]

emb_matrix = np.zeros((vocab_len, emb_dim))
for word, index in word_to_index.items():
    emb_matrix[index, :] = word_to_vec_map[word]


embedding_layer = layers.Embedding(vocab_len, emb_dim, input_length=sequence_length, trainable=False, weights=[emb_matrix])
embedding_layer.build((None,))
embedding_layer.set_weights([emb_matrix])

In [31]:
model = keras.Sequential([
                          layers.Embedding(vocab_len, emb_dim, input_length=sequence_length, trainable=False, weights=[emb_matrix]),
                          layers.LSTM(128, return_sequences=True),
                          layers.Dropout(0.5),
                          layers.LSTM(128, return_sequences=False),
                          layers.Dropout(0.5),
                          layers.Dense(5)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_6 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 20,223,927
Trainable params: 223,877
Non-trainable params: 20,000,050
____________________________________

In [32]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [33]:
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=20,
    min_delta=0.01
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Colab Notebooks/emoji_ckpt",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

In [34]:
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    shuffle=True
)

Epoch 1/50
5/5 [==============================] - 3s 21ms/step - loss: 1.6203 - accuracy: 0.2064
Epoch 2/50
5/5 [==============================] - 0s 21ms/step - loss: 1.5336 - accuracy: 0.3043
Epoch 3/50
5/5 [==============================] - 0s 20ms/step - loss: 1.4947 - accuracy: 0.3763
Epoch 4/50
5/5 [==============================] - 0s 20ms/step - loss: 1.4315 - accuracy: 0.4208
Epoch 5/50
5/5 [==============================] - 0s 22ms/step - loss: 1.3608 - accuracy: 0.4979
Epoch 6/50
5/5 [==============================] - 0s 19ms/step - loss: 1.2345 - accuracy: 0.6059
Epoch 7/50
5/5 [==============================] - 0s 20ms/step - loss: 1.1615 - accuracy: 0.5634
Epoch 8/50
5/5 [==============================] - 0s 20ms/step - loss: 1.0040 - accuracy: 0.6554
Epoch 9/50
5/5 [==============================] - 0s 21ms/step - loss: 0.8709 - accuracy: 0.6715
Epoch 10/50
5/5 [==============================] - 0s 19ms/step - loss: 0.8400 - accuracy: 0.7158
Epoch 11/50
5/5 [============

In [35]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 1s 9ms/step - loss: 0.5376 - accuracy: 0.8750


[0.5376462340354919, 0.875]

In [82]:
model.save("/content/drive/MyDrive/Colab Notebooks/emoji_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/emoji_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/emoji_model/assets


In [90]:
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/emoji_model")

In [91]:
def predict_emoji(sentence):
    tokens = tokenizer([sentence])
    emoji = np.argmax(model.predict(tokens))
    print(emojize(emoji_dictionary[emoji], use_aliases=True))

In [92]:
sentence = "It is a very difficult task to do"
predict_emoji(sentence)

😞


In [93]:
sentence = "I like her"
predict_emoji(sentence)

❤️


In [94]:
sentence = "Let us go outside and play a game"
predict_emoji(sentence)

⚾
